####  In this Notebook we are going to scrape the data from wikepedia page

Lets first install required libraraies

In [6]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libxml2: 2.9.4-h6b072ca_5     --> 2.9.8-h422b904_2     conda-forge
    libxslt: 1.1.29-hcf9102b_5    --> 1.1.32-h88dbc4e_2    conda-forge
    lxml:    4.1.0-py35ha401a81_0 --> 4.2.5-py35hc9114bc_0 conda-forge

libxml2-2.9.8- 100% |################################| Time: 0:00:00   8.54 MB/s
libxslt-1.1.32 100% |################################| Time: 0:00:00  62.90 MB/s
lxml-4.2.5-py3 100% |################################| Time: 0:00:00  27.53 MB/s


##### Lets import the BeautifulSoup ,requests and pandas library

In [ ]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd


Lets use the requests object to read the wikipedia page.
Pass the requests object to BeautifulSoup method (lxml parsing is used to parse the html page) which will retunr the soup object.
Using find_all method we can find the table. Use pandas read_html method to convert html table to list and then data frame.

In [41]:
res= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df_list = pd.read_html(str(table))
df=pd.DataFrame({'Postcode':df_list[0][0],'Borough':df_list[0][1],'Neighborhood':df_list[0][2]})
df=df.drop([0],axis=0).reset_index(drop=True)

Remove the rows where column value of Borough is 'Not assigned'

In [42]:
df=df[df.Borough != 'Not assigned'].reset_index(drop=True)
df.head()

,Borough,Neighborhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


1. Groupby the values based on 'Postcode' and 'Borough' and Concat the values of 'Neighborhood' in single row
2. Also Assign the value of'Borough' where 'Neighborhood' value is 'Not assigned'

In [44]:
df1=df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join)
df1=pd.DataFrame(df1).reset_index(drop=False)
for index,row in df1.iterrows():
   if row['Neighborhood']== 'Not assigned' :
     row['Neighborhood'] = row['Borough']

In [45]:
df1.shape[0]

103